## Libraries

In [ ]:
import numpy as np
import pandas as pd
import random

# Model GMM
from sklearn.mixture import GaussianMixture

#saves variables
import pickle 
#saves into files
from numpy import savetxt

# # MIC correlation
# from minepy import MINE

# # distance correlation
# import dcor
from scipy.spatial.distance import correlation


import matplotlib.pyplot  as plt
import seaborn as sns
# sns.set()

from keras import backend as K

# LSTM
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from scipy.stats import multivariate_normal
import scipy.stats as stats

In [ ]:
class ADClass:

    def __init__(self, dataset_path):
        """
        Initialize dataset file path.
        """
        self.dataset_path = dataset_path
        self.LCVs = []
        self.df_categorical_fields = ['MV101', 'P101', 'P102', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'P401', 'P402', 'P403', 'P404', 'UV401', 'P501', 'P502', 'P601', 'P602', 'P603']
        self.df_numerical_fields = ['FIT101', 'LIT101', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'DPIT301', 'FIT301', 'LIT301', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'PIT501', 'PIT502', 'PIT503', 'FIT601']
        

    def importDataset(self, file_name, nb_rows):
        if nb_rows == 0:
          return pd.read_excel(self.dataset_path + file_name, header=1)
        else:
          return pd.read_excel(self.dataset_path + file_name, header=1, nrows=nb_rows)

    def trimColumnName(slfe, df):

        return df.rename(columns=lambda x: x.strip())

    def splitDataset(self, df):
        x_n, y_n = df.iloc[:, :-1], df.iloc[:, [-1]]

        return  x_n, y_n

    def oneHotEncoding(self, fixed_value_list, x_n):
        new_cat_list = list(filter(lambda x: x not in fixed_value_list, self.df_categorical_fields))

        #creating instance of one-hot-encoder
        encoder = OneHotEncoder(handle_unknown='ignore')
        # One-hot-encode the categorical columns.
        enc_package_type = pd.DataFrame(encoder.fit_transform(x_n[new_cat_list]).toarray())
        #merge one-hot encoded columns back with original DataFrame
        x_n = x_n.join(enc_package_type)
        
        return x_n

    def removeCategorical(self, fixed_value_list, x_n):
        new_cat_list = list(filter(lambda x: x not in fixed_value_list, self.df_categorical_fields))
        x_n.drop(new_cat_list, axis=1, inplace=True)

        return x_n


    def createSequence(slef, x_n_scaled, window_size, shift):
        # Create sequences
        train_windows = []
        for i in range(0, x_n_scaled.shape[0] - window_size - shift, shift):
            train_windows.append(x_n_scaled[i:i+window_size])

        # Convert to numpy array
        x_w_train = np.array(train_windows)

        return x_w_train

    def windowDetectionLSTM(self, window_size_list, lstm_X_train, lstm_X_test, LSTM_w_shift):
        history_list = []
        mse_test = []

        for i in window_size_list:
          window_size = i

          # Create sequences
          train_windows = []
          test_windows = []

          x_w_train = AD_normal.createSequence(lstm_X_train, window_size, LSTM_w_shift)
          x_w_test = AD_normal.createSequence(lstm_X_test, window_size, LSTM_w_shift)

          # data dimensions // hyperparameters 
          input_dim, output_dim = x_w_train.shape, x_w_train.shape[2]
          BATCH_SIZE = 128
          EPOCHS = 50

          # # https://keras.io/layers/core/
          # LSTM_model = keras.models.Sequential([
              
          #     # deconstruct / encode
          #     keras.layers.Dense(input_dim, activation='elu', input_shape=(x_w_train.shape[1], x_w_train.shape[2])), 
          #     keras.layers.Dense(16, activation='elu'),
          #     keras.layers.Dense(8, activation='elu'),
          #     keras.layers.Dense(4, activation='elu'),
          #     keras.layers.Dense(2, activation='elu'),
              
          #     # reconstruction / decode
          #     keras.layers.Dense(4, activation='elu'),
          #     keras.layers.Dense(8, activation='elu'),
          #     keras.layers.Dense(16, activation='elu'),
          #     keras.layers.Dense(input_dim, activation='elu')
              
          # ])

          LSTM_model = keras.models.Sequential([
              keras.layers.InputLayer(input_shape=(x_w_train.shape[1], x_w_train.shape[1], x_w_train.shape[2])),
              keras.layers.LSTM(32),
              keras.layers.Dense(16),
              keras.layers.Dense(16),
              keras.layers.Lambda(lambda x: x[0] + x[1] * K.random_normal(shape=(K.shape(x[1])[0], 16))),
              keras.layers.RepeatVector(4),
              keras.layers.LSTM(32, return_sequences=True),
              keras.layers.Dense(output_dim)
          ])

          # # define our early stopping
          # early_stop = keras.callbacks.EarlyStopping(
          #     monitor='val_loss',
          #     min_delta=0.0001,
          #     patience=10,
          #     verbose=1, 
          #     mode='min',
          #     restore_best_weights=True
          # )


          # the default learning rate is used for the Adam optimizer, which is typically set to 0.001. 
          LSTM_model.compile(optimizer="adam", 
                              loss="mse",
                              metrics=["acc"])

          # print an overview of our model
          LSTM_model.summary();

          history = LSTM_model.fit(
            x_w_train,
            x_w_train,
            epochs=50,
            batch_size=128,
            validation_split=0.2,
            callbacks=[
                keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
            ],
          )

          plt.plot(history.history["loss"], label="Training Loss")
          plt.plot(history.history["val_loss"], label="Validation Loss")
          plt.legend()
          plt.show()

          # Generate predictions on the test set
          X_w_pred = LSTM_model.predict(x_w_test)

          # Evaluate the autoencoder
          mse = np.mean(np.power(x_w_test - X_w_pred, 2), axis=1)
          print("Mean Squared Error:", np.mean(mse))
          print()
          
          # save values 
          history_list.append(history)
          mse_test.append(np.mean(mse))
        
        return history_list, mse_test


    # /*
    # transposing the window array is necessary to ensure that each row represents a
    # feature and each column represents a sample, which is the correct format for 
    # calculating the correlation matrix using the np.corrcoef function.

    # NOTE:
    # -----
    # Solutions to avoin NaN from deviding by zero
    # 1- To avoid the divide-by-zero problem when dealing with constant values. 
    #    One such coefficient is the Spearman rank correlation coefficient.
    # 2- Add small amount of noise to the data.
    # */
    def calculateLCV(self, x_w_train):

        LCVs = []

        ## 
        # for i in range(x_w_train.shape[0]):
        #   corr_matrix = np.corrcoef(x_w_train[i].T)
        #   LCMs.append(corr_matrix)

        ## sol.1 :
        # for i in range(x_w_train.shape[0]):
        #   window_df = pd.DataFrame(x_w_train[i])
        #   rank_df = window_df.rank(method='average')
        #   corr_matrix = rank_df.corr(method='pearson')
        #   LCMs.append(corr_matrix)

        # sol.2 :
        for i in range(x_w_train.shape[0]):
          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_w_train[i])
         
          # find constant columns using NumPy isclose() function
          constant_cols = [col for col in window_df.columns if np.isclose(window_df[col], window_df[col].iloc[0], rtol=1e-15, atol=1e-15).all()]

          # Print the constant columns in the window
          # print(f"Constant columns in 2nd window {i//window_size}: {constant_cols}")

          # Add a small amount of random noise to the constant columns
          window_df[constant_cols] += np.abs(np.random.normal(scale=0.00000001, size=window_df[constant_cols].shape))
         
          # Compute the correlation matrix for the window
          corr_matrix = window_df.corr()

          # Extract the upper triangle of the correlation matrix into a vector
          correlation_vector = corr_matrix.values[np.triu_indices_from(corr_matrix, k=1)]

          ### DEBUG
          # print(window_df)
          # print(corr_matrix)
          # print(correlation_vector)

          # from numpy import savetxt
          # # save to csv file
          # savetxt('window_df.csv', window_df, delimiter=',')
          # savetxt('corr_matrix_.csv', corr_matrix, delimiter=',')
          # savetxt('correlation_vector_.csv', correlation_vector, delimiter=',')

          LCVs.append(correlation_vector)
          if not len(LCVs)%100:
            print(len(LCVs),'\n')

        return LCVs
    
    def calculateLCV_kendall(self, x_w_train, path):

        LCVs = []

      
        for i in range(x_w_train.shape[0]):
          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_w_train[i])
         
          # # find constant columns using NumPy isclose() function
          # constant_cols = [col for col in window_df.columns if np.isclose(window_df[col], window_df[col].iloc[0], rtol=1e-15, atol=1e-15).all()]

          # # Print the constant columns in the window
          # # print(f"Constant columns in 2nd window {i//window_size}: {constant_cols}")

          # # Add a small amount of random noise to the constant columns
          # window_df[constant_cols] += np.abs(np.random.normal(scale=0.00000001, size=window_df[constant_cols].shape))
         
          # Compute the correlation matrix for the window
          corr_matrix = window_df.corr(method='kendall')

          # Extract the upper triangle of the correlation matrix into a vector
          correlation_vector = corr_matrix.values[np.triu_indices_from(corr_matrix, k=1)]

          LCVs.append(correlation_vector)
          if not len(LCVs)%100:
            print(len(LCVs),'\n')
          
          if len(LCVs) % 1000 == 0:
              %cd $training_path

        return LCVs

    def calculateLCV_spearman(self, x_w_train):

        LCVs = []

      
        for i in range(x_w_train.shape[0]):
          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_w_train[i])
         
          # find constant columns using NumPy isclose() function
          constant_cols = [col for col in window_df.columns if np.isclose(window_df[col], window_df[col].iloc[0], rtol=1e-15, atol=1e-15).all()]

          # Print the constant columns in the window
          # print(f"Constant columns in 2nd window {i//window_size}: {constant_cols}")

          # Add a small amount of random noise to the constant columns
          window_df[constant_cols] += np.abs(np.random.normal(scale=0.00000001, size=window_df[constant_cols].shape))
         
          # Compute the correlation matrix for the window
          corr_matrix = window_df.corr(method='spearman')

          # Extract the upper triangle of the correlation matrix into a vector
          correlation_vector = corr_matrix.values[np.triu_indices_from(corr_matrix, k=1)]

          LCVs.append(correlation_vector)
          if not len(LCVs)%100:
            print(len(LCVs),'\n')

        return LCVs
      
    def checkPosDef(self, x):
          
        return np.all(np.linalg.eigvals(x) > 0)

    def MGD(self, mean_vector, covariance_matrix):

        mgd = multivariate_normal(mean=mean_vector, cov=covariance_matrix)

        return mgd

    def saveFile(self, file_name, var):
        # save to csv file
        savetxt(file_name, var, delimiter=',')
        
        return

    def calculateLCV_MIC(self, x_w_train):

        LCVs = []

        for k in range(x_w_train.shape[0]):

          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_w_train[k])
          cols = window_df.columns
          n_cols = len(cols)

          my_list = []

          for i in range(n_cols):
              for j in range(i+1, n_cols):
                  col1, col2 = cols[i], cols[j]
                  mine = MINE()
                  mine.compute_score(window_df[col1], window_df[col2])
                  my_list.append(mine.mic())
          
          my_array = np.array(my_list)

          LCVs.append(my_array)

          if not len(LCVs)%100:
            print(len(LCVs),'\n')

        return LCVs

    def calculateLCV_DistnaceCorr(self, x_w_train):

  
        LCVs = []
        n_cols = x_w_train.shape[2]
        
        for k in range(x_w_train.shape[0]):
            # Convert the window to a Pandas DataFrame
            window_df = pd.DataFrame(x_w_train[k])
            my_list = []

            for i in range(n_cols):
                for j in range(i+1, n_cols):
                    col1, col2 = window_df.columns[i], window_df.columns[j]
                    dcor_ = dcor.distance_correlation(window_df[col1], window_df[col2])
                    # dcor_ = correlation(window_df[col1], window_df[col2], w=None, centered=True)
                    my_list.append(dcor_)

            my_array = np.array(my_list)
            LCVs.append(my_array)

            if len(LCVs) % 100 == 0:
                print(len(LCVs), '\n')

        return LCVs

    def calculateLCV_kendalltau(self, x_w_train):
  
        LCVs = []
        n_cols = x_w_train.shape[2]
        
        for k in range(x_w_train.shape[0]):
            # Convert the window to a Pandas DataFrame
            window_df = pd.DataFrame(x_w_train[k])
            my_list = []

            for i in range(n_cols):
                for j in range(i+1, n_cols):
                    col1, col2 = window_df.columns[i], window_df.columns[j]
                    tau, p_value = stats.kendalltau(window_df[col1], window_df[col2])                   
                    my_list.append(tau)

            my_array = np.array(my_list)
            LCVs.append(my_array)

            if len(LCVs) % 100 == 0:
                print(len(LCVs), '\n')

        return LCVs

    def getLabels(self, x_y_train):

        LCVs_y = []

        for i in range(x_y_train.shape[0]):
          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_y_train[i])       

          # Define the value you want to check for
          values_to_check = ['Attack', 'A ttack']

          # Check if the value exists in the 'B' column
          if window_df[0].isin(values_to_check).any():
              # print(window_df[1].values)
              LCVs_y.append(1)
          else:
              # print(window_df[0].values)
              LCVs_y.append(0)
          

        return LCVs_y 

    def getBinaryLabels(self, y_labels):
        # Initialize an empty list
        y_labels_bin = []

        # Define the values you want to check for
        values_to_check = ['Attack', 'A ttack']

        # Iterate over the cells in the column
        for cell in y_labels['Normal/Attack']:
            if cell in values_to_check:
                y_labels_bin.append(1)
            else:
                y_labels_bin.append(0)

        # Convert the list to a DataFrame
        y_labels_bin_df = pd.DataFrame({'isAttack': y_labels_bin})

        return y_labels_bin_df

    def select_threshold(self, probs, test_data):
        best_epsilon = 0
        best_f1 = 0
        f = 0
        stepsize = (max(probs) - min(probs)) / 2000;
        epsilons = np.arange(min(probs), max(probs), stepsize)
        for epsilon in np.nditer(epsilons):
            predictions = (probs < epsilon)
            predictions
            f = f1_score(test_data, predictions, average='binary')
            # r = recall_score(test_data, predictions, average='binary')
            # Create confusion matrix
            
            if f > best_f1:
                best_f1 = f
                best_epsilon = epsilon

        # confusion matrix
        predictions = (probs < best_epsilon)
        cm = confusion_matrix(test_data, predictions)

        return best_f1, best_epsilon, cm

    def predictWithEpsilon(self, X, mvn, best_epsilon):
        predictions = []

        pdf = mvn.logpdf(X)
        predictions = (pdf < best_epsilon)  
        
        return predictions, pdf

    def predictWithAlpha(self, LCV_list, Sigma, mu, alpha):
        predictions = []

        lower_bound = mu - alpha * np.diag(Sigma)
        upper_bound = mu + alpha * np.diag(Sigma)

        for LCV in LCV_list:
          if (mu - alpha * np.diag(Sigma) <= LCV).all() and (LCV <= mu + alpha * np.diag(Sigma)).all():
              predictions.append(0)
          elif (LCV < lower_bound).any() or (LCV > upper_bound).any():
              predictions.append(1)          
        
        return predictions

    def findBestAlpha(self, LCVs, ground_truth, Sigma, mu):
        
        best_f1 = 0
        best_alpha = 0
        best_cm = None
        
        alpha = 0
        step = 1
        # for alpha in range(0, 50, 0.2):
        while alpha < 250:
            predictions = self.predictWithAlpha(LCVs, Sigma, mu, alpha)
            f1 = f1_score(ground_truth, predictions, average='binary')
            
            if f1 > best_f1:
                best_f1 = f1
                best_alpha = alpha

            alpha += step

        # confusion matrix
        predictions = self.predictWithAlpha(LCVs, Sigma, mu, best_alpha)
        cm = confusion_matrix(ground_truth, predictions)

        return best_f1, best_alpha, cm


In [ ]:
# probs = [2 ,2,4,4,5,223,12,234,345,2]

# probs = np.array(probs)
# epsilon  = 10
# predictions = (probs < epsilon)

# test_data = [1 ,1,1,1,1,0,0,0,0,0]
# # [ True  True  True  True  True False False False False  True]
# print(predictions)
# f = recall_score(test_data, predictions, average='binary')


# # Create confusion matrix
# cm = confusion_matrix(test_data, predictions)

# # Plot confusion matrix using seaborn
# sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", cbar=False)
# plt.xlabel("Predicted Labels")
# plt.ylabel("True Labels")
# plt.show()

## Import Data

24 Sep 18 (SWaT.A2_Dec 2015)
Two sets of “SWaT_Dataset_Normal” – versions 0 and 1 – are provided. The datasets capture the normal state of the SWaT testbed running for seven days. In Version 0, we started recording the data when the plant was emptying the water storage tank for 30 minutes. In general, in an ICS environment, this is part of the maintenance outside normal operations. As a result of this drainage, the first 30 minutes of LIT101 data exhibits change even though there was no water in/outflow. Version 1 is derived from version 0 by removing the first 30 minutes of data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/Thesis/'
%pwd
# from AD_Class import ADClass

/content/drive/My Drive/Colab Notebooks/Thesis


'/content/drive/My Drive/Colab Notebooks/Thesis'

In [ ]:
dataset_path = 'SWaT.A1 & A2_Dec 2015/Physical/'
# dataset_path = 'dataset/'

SWaT_attack = 'SWaT_Dataset_Attack_v0.xlsx'
SWaT_normal_v0 = 'SWaT_Dataset_Normal_v0.xlsx'
SWaT_normal_v1 = 'SWaT_Dataset_Normal_v1.xlsx'

In [ ]:
# del AD_normal

In [ ]:
AD_normal = ADClass(dataset_path)
AD_attack = ADClass(dataset_path)

In [ ]:
df = AD_normal.importDataset(file_name = SWaT_normal_v1, nb_rows = 0)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
df_an = AD_attack.importDataset(file_name = SWaT_attack, nb_rows = 0)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
# df.iloc[:,:12]
df

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,22/12/2015 4:30:00 PM,0.000000,124.3135,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,Normal
1,22/12/2015 4:30:01 PM,0.000000,124.3920,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,Normal
2,22/12/2015 4:30:02 PM,0.000000,124.4705,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,Normal
3,22/12/2015 4:30:03 PM,0.000000,124.6668,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,Normal
4,22/12/2015 4:30:04 PM,0.000000,124.5098,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494995,28/12/2015 9:59:55 AM,2.460366,523.0430,2,2,1,262.0161,8.396437,328.5055,2.442316,...,2,1,250.817100,1.778105,189.8552,0.000128,1,1,1,Normal
494996,28/12/2015 9:59:56 AM,2.448836,522.9645,2,2,1,262.0161,8.396437,328.5055,2.442316,...,2,1,250.817100,1.778105,189.5027,0.000128,1,1,1,Normal
494997,28/12/2015 9:59:57 AM,2.434744,522.8860,2,2,1,262.0161,8.396437,328.6337,2.444879,...,2,1,250.817100,1.778105,189.5027,0.000128,1,1,1,Normal
494998,28/12/2015 9:59:58 AM,2.428338,522.9252,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.817100,1.649953,189.5027,0.000128,1,1,1,Normal


In [ ]:
df_an

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,28/12/2015 10:00:01 AM,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,28/12/2015 10:00:03 AM,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,28/12/2015 10:00:04 AM,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449914,2/1/2016 2:59:55 PM,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,...,2,1,251.1535,0.865024,189.0220,0.000000,1,1,1,Normal
449915,2/1/2016 2:59:56 PM,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal
449916,2/1/2016 2:59:57 PM,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal
449917,2/1/2016 2:59:58 PM,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal


### remove spaces from columns name

In [ ]:
df = AD_normal.trimColumnName(df)

In [ ]:
df_an = AD_attack.trimColumnName(df_an)

### make copy

In [ ]:
# df_or = df.copy()

In [ ]:
# df_or_an = df_an.copy()

In [ ]:
# df = df_or.copy()

### Split data

In [ ]:
# Split data from labels
x_n, y_n = AD_normal.splitDataset(df)

In [ ]:
x_n_an, y_n_an = AD_attack.splitDataset(df_an)

In [ ]:
# remove date column
x_n.drop('Timestamp', axis=1, inplace=True)

In [ ]:
x_n_an.drop('Timestamp', axis=1, inplace=True)

# Pre-processing

### - Removing constant fields

In [ ]:
fixed_value_fields = ['P202', 'P401', 'P404', 'P502', 'P601', 'P603']

In [ ]:
x_n.drop(fixed_value_fields, axis=1, inplace=True)

In [ ]:
x_n_an.drop(fixed_value_fields, axis=1, inplace=True)

In [ ]:
x_n.shape

(495000, 45)

In [ ]:
x_n_an.shape

(449919, 45)

### Merge all before one-hot encoding

In [ ]:
merged_df = pd.concat([x_n, x_n_an], axis=0, ignore_index=True)

In [ ]:
merged_labels = pd.concat([y_n, y_n_an], axis=0, ignore_index=True)

In [ ]:
merged_labels_bin = AD_normal.getBinaryLabels(merged_labels)

In [ ]:
# to fix dimention 1D
merged_labels_bin = np.ravel(merged_labels_bin)

## one-hot encoding

In [ ]:
merged_df = AD_normal.oneHotEncoding(fixed_value_fields, merged_df)

In [ ]:
merged_df.shape

(944919, 91)

In [ ]:
merged_df.columns

Index([ 'FIT101',  'LIT101',   'MV101',    'P101',    'P102',  'AIT201',
        'AIT202',  'AIT203',  'FIT201',   'MV201',    'P201',    'P203',
          'P204',    'P205',    'P206', 'DPIT301',  'FIT301',  'LIT301',
         'MV301',   'MV302',   'MV303',   'MV304',    'P301',    'P302',
        'AIT401',  'AIT402',  'FIT401',  'LIT401',    'P402',    'P403',
         'UV401',  'AIT501',  'AIT502',  'AIT503',  'AIT504',  'FIT501',
        'FIT502',  'FIT503',  'FIT504',    'P501',  'PIT501',  'PIT502',
        'PIT503',  'FIT601',    'P602',         0,         1,         2,
               3,         4,         5,         6,         7,         8,
               9,        10,        11,        12,        13,        14,
              15,        16,        17,        18,        19,        20,
              21,        22,        23,        24,        25,        26,
              27,        28,        29,        30,        31,        32,
              33,        34,        35,        36, 

In [ ]:
# merged_df[merged_df.index < 5]

In [ ]:
# x_n = AD_normal.oneHotEncoding(fixed_value_fields, x_n)

In [ ]:
# x_n_an = AD_attack.oneHotEncoding(fixed_value_fields, x_n_an)

In [ ]:
# x_n.shape

## Split all after one-hot encoding

In [ ]:
# Split back into original DataFrames
x_n = merged_df[merged_df.index < len(x_n)]
x_n_an = merged_df[merged_df.index >= len(x_n)]

In [ ]:
x_n

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,36,37,38,39,40,41,42,43,44,45
0,0.000000,124.3135,1,1,1,251.9226,8.313446,312.7916,0.000000,1,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.000000,124.3920,1,1,1,251.9226,8.313446,312.7916,0.000000,1,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.000000,124.4705,1,1,1,251.9226,8.313446,312.7916,0.000000,1,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.000000,124.6668,1,1,1,251.9226,8.313446,312.7916,0.000000,1,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.000000,124.5098,1,1,1,251.9226,8.313446,312.7916,0.000000,1,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494995,2.460366,523.0430,2,2,1,262.0161,8.396437,328.5055,2.442316,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494996,2.448836,522.9645,2,2,1,262.0161,8.396437,328.5055,2.442316,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494997,2.434744,522.8860,2,2,1,262.0161,8.396437,328.6337,2.444879,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494998,2.428338,522.9252,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [ ]:
# Reset the index
x_n_an.reset_index(drop=True, inplace=True)

In [ ]:
x_n_an

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,36,37,38,39,40,41,42,43,44,45
0,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449914,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
449915,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
449916,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
449917,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [ ]:
del merged_df

## Remvoe categoriacl fields after encoding

In [ ]:
x_n = AD_normal.removeCategorical(fixed_value_fields, x_n)

<ipython-input-3-626204a77f1d>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_n.drop(new_cat_list, axis=1, inplace=True)


In [ ]:
x_n_an = AD_attack.removeCategorical(fixed_value_fields, x_n_an)

In [ ]:
x_n.columns = x_n.columns.astype(str)

In [ ]:
x_n_an.columns = x_n_an.columns.astype(str)

In [ ]:
x_n.columns

Index(['FIT101', 'LIT101', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'DPIT301',
       'FIT301', 'LIT301', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'AIT501',
       'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504',
       'PIT501', 'PIT502', 'PIT503', 'FIT601', '0', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41',
       '42', '43', '44', '45'],
      dtype='object')

In [ ]:
x_n_an.columns

Index(['FIT101', 'LIT101', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'DPIT301',
       'FIT301', 'LIT301', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'AIT501',
       'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504',
       'PIT501', 'PIT502', 'PIT503', 'FIT601', '0', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41',
       '42', '43', '44', '45'],
      dtype='object')

In [ ]:
x_n

,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,AIT401,...,36,37,38,39,40,41,42,43,44,45
0,0.000000,124.3135,251.9226,8.313446,312.7916,0.000000,2.560983,0.000256,138.5061,0.000,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.000000,124.3920,251.9226,8.313446,312.7916,0.000000,2.560983,0.000256,138.7465,0.000,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.000000,124.4705,251.9226,8.313446,312.7916,0.000000,2.560983,0.000256,138.6263,0.000,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.000000,124.6668,251.9226,8.313446,312.7916,0.000000,2.560983,0.000256,138.7064,0.000,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.000000,124.5098,251.9226,8.313446,312.7916,0.000000,2.560983,0.000256,138.9067,0.000,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494995,2.460366,523.0430,262.0161,8.396437,328.5055,2.442316,19.748380,2.212087,955.8046,148.808,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494996,2.448836,522.9645,262.0161,8.396437,328.5055,2.442316,19.748380,2.210037,955.8847,148.808,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494997,2.434744,522.8860,262.0161,8.396437,328.6337,2.444879,19.748380,2.207731,955.9648,148.808,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494998,2.428338,522.9252,262.0161,8.396437,328.6337,2.445391,19.748380,2.206835,956.2051,148.808,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


## Standardization

In [ ]:
scaler_std = StandardScaler()
scaler_std.fit(x_n)
x_n_scaled = scaler_std.transform(x_n)


In [ ]:
scaler_std = StandardScaler()
scaler_std.fit(x_n_an)
x_n_an_scaled = scaler_std.transform(x_n_an)

In [ ]:
print(x_n_scaled)

[[ -1.63398414  -3.80729131  -2.47768267 ... -17.91219009   0.08948205
   -0.08948205]
 [ -1.63398414  -3.8066461   -2.47768267 ... -17.91219009   0.08948205
   -0.08948205]
 [ -1.63398414  -3.8060009   -2.47768267 ... -17.91219009   0.08948205
   -0.08948205]
 ...
 [  0.51586549  -0.53134468  -0.36920891 ...   0.0558279    0.08948205
   -0.08948205]
 [  0.51020907  -0.53102249  -0.36920891 ...   0.0558279    0.08948205
   -0.08948205]
 [  0.50907796  -0.5316677   -0.36920891 ...   0.0558279    0.08948205
   -0.08948205]]


## Normalization


In [ ]:
# Normalization the data using MinMaxScaler
scaler_minmax  = MinMaxScaler()

In [ ]:
scaler_minmax.fit(x_n)
x_n_scaled = scaler_minmax.transform(x_n)

In [ ]:
print(x_n_scaled)

[[0.         0.00529434 0.01229141 ... 0.         1.         0.        ]
 [0.         0.00540698 0.01229141 ... 0.         1.         0.        ]
 [0.         0.00551961 0.01229141 ... 0.         1.         0.        ]
 ...
 [0.88694441 0.57718951 0.49615774 ... 1.         1.         0.        ]
 [0.88461079 0.57724575 0.49615774 ... 1.         1.         0.        ]
 [0.88414414 0.57713312 0.49615774 ... 1.         1.         0.        ]]


In [ ]:
# Normalization the data using MinMaxScaler
scaler_minmax.fit(x_n_an)
x_n_an_scaled = scaler_minmax.transform(x_n_an)

In [ ]:
# savetxt('x_n.csv', x_n.iloc[:10000,:], delimiter=',')


In [ ]:
x_n_scaled.shape

(495000, 71)

In [ ]:
# Convert ndarray to pandas DataFrame
df_x_n_scaled = pd.DataFrame(x_n_scaled)
df_x_n_an_scaled = pd.DataFrame(x_n_an_scaled)

In [ ]:
df_x_n_scaled

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,0.000000,0.005294,0.012291,0.075099,0.002009,0.000000,0.121378,0.000109,0.006449,0.000000,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.000000,0.005407,0.012291,0.075099,0.002009,0.000000,0.121378,0.000109,0.006722,0.000000,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.000000,0.005520,0.012291,0.075099,0.002009,0.000000,0.121378,0.000109,0.006586,0.000000,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.000000,0.005801,0.012291,0.075099,0.002009,0.000000,0.121378,0.000109,0.006676,0.000000,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.000000,0.005576,0.012291,0.075099,0.002009,0.000000,0.121378,0.000109,0.006903,0.000000,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494995,0.896278,0.577415,0.496158,0.188845,0.063586,0.981663,0.935973,0.937812,0.933191,0.999677,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494996,0.892078,0.577302,0.496158,0.188845,0.063586,0.981663,0.935973,0.936943,0.933282,0.999677,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494997,0.886944,0.577190,0.496158,0.188845,0.064088,0.982693,0.935973,0.935965,0.933372,0.999677,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
494998,0.884611,0.577246,0.496158,0.188845,0.064088,0.982899,0.935973,0.935586,0.933645,0.999677,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


feature selection

In [ ]:
y_n_bin = AD_normal.getBinaryLabels(y_n)

In [ ]:
# to fix dimention 1D
y_n_bin = np.ravel(y_n_bin)

In [ ]:
y_n_an_bin = AD_normal.getBinaryLabels(y_n_an)

In [ ]:
# to fix dimention 1D
y_n_an_bin = np.ravel(y_n_an_bin)

In [ ]:
from sklearn.feature_selection import SelectPercentile, mutual_info_classif

# Assuming X contains your feature matrix and y contains the target variable

# Define the percentage of features to select
percentile = 10

# Calculate mutual information scores
mi_scores = mutual_info_classif(df_x_n_an_scaled.iloc[:,:], y_n_an_bin)

# Select top percentile features based on MI scores
selector = SelectPercentile(mutual_info_classif, percentile=percentile)
selected_features = selector.fit_transform(df_x_n_an_scaled.iloc[:,:], y_n_an_bin)

# Get the indices of selected features
selected_feature_indices = selector.get_support(indices=True)

# Print the selected feature indices
print("Selected feature indices:", selected_feature_indices)

In [ ]:
df_x_n_an_scaled_MI = df_x_n_an_scaled.iloc[:, selected_feature_indices]

# Model build

In [ ]:
from sklearn.model_selection import train_test_split
  
X_train, X_test, y_train, y_test = train_test_split( df_x_n_an_scaled_MI, y_n_an_bin,  test_size = 0.20, random_state = 101)

In [ ]:
  # defining parameter range
param_grid = {'C': [0.1, 1, 10], 
              'penalty': ['l1','l2']} 
  
grid = GridSearchCV(LinearSVC(), param_grid, refit = True, verbose = 1)

In [ ]:
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


GridSearchCV(estimator=LinearSVC(),
             param_grid={'C': [0.1, 1, 10], 'penalty': ['l2']}, verbose=1)

In [ ]:
# Predict the anomalies
prediction = grid.predict(X_test)


In [ ]:
value_counts = np.bincount(prediction)

print(value_counts)

[124924  10052]


In [ ]:
# # Change the anomalies' values to make it consistent with the true values
# prediction = [1 if i==-1 else 0 for i in prediction]

In [ ]:
from sklearn.metrics import classification_report
# Check the model performance
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97    118539
           1       1.00      0.61      0.76     16437

    accuracy                           0.95    134976
   macro avg       0.97      0.80      0.86    134976
weighted avg       0.95      0.95      0.95    134976

